In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from matplotlib.patches import Circle
import os
import pickle
from scipy.stats import mannwhitneyu, ttest_ind, kruskal, f_oneway
from statannotations.Annotator import Annotator
from itertools import combinations
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output, display

In [3]:
df_food_scaled = pd.read_csv('df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_18236\4074791732.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('df_food_scaled.csv')


In [4]:
nutrient_info = {
    'folate_eaten':                                         {'unit': 'mcg', 'target': 400,  'type': 'qualifying'},
    'niacin_eaten':                                         {'unit': 'mg',  'target': 14,   'type': 'qualifying'},
    'pantothenic_acid_eaten':                               {'unit': 'mg',  'target': 5,    'type': 'qualifying'},
    'vitamin_b2_eaten':                                     {'unit': 'mg',  'target': 1.1,  'type': 'qualifying'},
    'vitamin_b1_eaten':                                     {'unit': 'mg',  'target': 1.2,  'type': 'qualifying'},
    'all_trans_retinol_equivalents_activity_eaten':         {'unit': 'IU',  'target': 700,  'type': 'qualifying'},
    'beta_carotene_eaten':                                  {'unit' : 'mcg','target' : 700, 'type' :'qualifying'},
    'vitamin_b12_eaten':                                    {'unit': 'mcg', 'target': 2.4,  'type': 'qualifying'},
    'vitamin_b6_eaten':                                     {'unit': 'mg',  'target': 1.3,  'type': 'qualifying'},
    'vitamin_c_eaten':                                      {'unit': 'mg',  'target': 75,   'type': 'qualifying'},
    'vitamin_d_eaten':                                      {'unit': 'mcg', 'target': 7.5,  'type': 'qualifying'},
    'vitamin_e_activity_eaten':                             {'unit': 'TAE', 'target': 15,    'type': 'qualifying'},
    'calcium_eaten':                                        {'unit': 'mg',  'target': 1000, 'type': 'qualifying'},
    'iron_eaten':                                           {'unit': 'mg',  'target': 18,   'type': 'qualifying'},
    'magnesium_eaten':                                      {'unit': 'mg',  'target': 320,  'type': 'qualifying'},
    'phosphorus_eaten':                                     {'unit': 'mg',  'target': 700,  'type': 'qualifying'},
    'potassium_eaten':                                      {'unit': 'mg',  'target': 4700, 'type': 'qualifying'},
    'zinc_eaten':                                           {'unit': 'mg',  'target': 8,    'type': 'qualifying'},
    'fiber_eaten':                                          {'unit': 'g',   'target': 25,   'type': 'qualifying'},
    'protein_eaten':                                        {'unit': 'g',   'target': 46,   'type': 'qualifying'},
    'fat_eaten':                                            {'unit': 'g',   'target': 78,   'type': 'disqualifying'},
    'fatty_acids_saturated_eaten':                          {'unit': 'g',   'target': 22,   'type': 'disqualifying'},
    'cholesterol_eaten':                                    {'unit': 'mg',  'target': 300,  'type': 'disqualifying'},
    'sugar_eaten':                                          {'unit': 'g',   'target': 125,  'type': 'disqualifying'},
    'sodium_eaten':                                         {'unit': 'mg',  'target': 2400, 'type': 'disqualifying'},
    'salt_eaten' :                                          {'unit' : 'g',  'target': 6,   'type': 'disqualifying'}
}


conversion_factors = {
    "mg": 1000,
    "g": 1,
    "mcg": 1000000,
    "IU": 1,
    "TAE": 1,
}

In [5]:
qualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'qualifying'}
disqualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'disqualifying'}

In [6]:
df_vegetable_fruit_juice = df_food_scaled[df_food_scaled['food_group'] == 'vegetable_fruit_juices']
df_vegetable_fruit_juice.drop_duplicates('combined_name')[['combined_name', 'QI', 'DI', 'NB']].sort_values('QI', ascending=False)

,combined_name,QI,DI,NB
293872,COOP NATURAPLAN BIO Biotta Gemüse-Cocktail,30.366425,1.928472,84.166667
298929,Migros ORGANIC Vegetable Cocktail,29.095072,2.520723,84.000000
300997,Jus de curcuma bio,22.297306,0.285589,77.944269
195496,"Juice, vegetable",22.078085,0.716485,81.429513
302968,Alnatura beetroot juice,20.217955,0.807460,81.250000
...,...,...,...,...
316659,Innocent Direktsaft Apfel & Mango,0.000000,0.521739,0.000000
316978,Coop Naturaplan Bio - Zitronensaft,0.000000,0.185507,0.000000
316889,Rabenhorst - pomegranate,0.000000,0.612022,0.000000
317505,demeter - Karottensaft (Rodelika),0.000000,0.696477,0.000000


In [7]:
high_qi_vegetable_fruit_juices = df_vegetable_fruit_juice[df_vegetable_fruit_juice['QI'] > 3].drop_duplicates('combined_name').sort_values('QI', ascending=False)

In [8]:
high_qi_vegetable_fruit_juices[['combined_name', 'QI', 'DI', 'NB']]

,combined_name,QI,DI,NB
293872,COOP NATURAPLAN BIO Biotta Gemüse-Cocktail,30.366425,1.928472,84.166667
298929,Migros ORGANIC Vegetable Cocktail,29.095072,2.520723,84.000000
300997,Jus de curcuma bio,22.297306,0.285589,77.944269
195496,"Juice, vegetable",22.078085,0.716485,81.429513
302968,Alnatura beetroot juice,20.217955,0.807460,81.250000
305192,coop naturaplan Randensaft,17.698268,0.655962,75.487805
292291,Pfanner Bio - Organic beetroot,16.958814,0.823300,80.232558
292435,"Juicy: pear, orange, banana, spinach, celeriac",10.360972,0.460064,72.547653
303801,Coop Naturaplan Sweet Mild Orange Mild,9.831389,0.585366,12.529162
294697,Hohes C - Milder Multivitamin 100% Saft,8.248337,0.676611,74.824992


In [9]:
ratio_cols = [i for i in high_qi_vegetable_fruit_juices.columns if '_ratio_scaled' in i]

df_corr = high_qi_vegetable_fruit_juices[ratio_cols + ['QI']]
corr = df_corr.corr()['QI']
corr.sort_values(ascending=True)

vitamin_b1_eaten_ratio_scaled                               -0.446987
sugar_eaten_ratio_scaled                                    -0.424316
niacin_eaten_ratio_scaled                                   -0.405232
vitamin_b2_eaten_ratio_scaled                               -0.364504
vitamin_e_activity_eaten_ratio_scaled                       -0.229465
vitamin_b12_eaten_ratio_scaled                              -0.183108
vitamin_c_eaten_ratio_scaled                                -0.163494
vitamin_d_eaten_ratio_scaled                                -0.109932
vitamin_b6_eaten_ratio_scaled                               -0.080208
folate_eaten_ratio_scaled                                   -0.068749
sodium_eaten_ratio_scaled                                   -0.006292
pantothenic_acid_eaten_ratio_scaled                          0.121522
fat_eaten_ratio_scaled                                       0.234113
zinc_eaten_ratio_scaled                                      0.256409
salt_eaten_ratio_sca

In [10]:
def compute_index(row, nutrient_cols, scaling_factor=2000) :

    index = 0
    ratio_sum = 0

    for nutr in nutrient_cols :
        ratio_sum += row[nutr + '_ratio']

    index = (scaling_factor / row['energy_kcal_eaten']) * (ratio_sum / len(nutrient_cols))
    return index

In [11]:
def compute_qi_excluding(row, nutrient_list, exclude=None, scaling_factor=2000):
    if exclude is not None:
        new_list = [nutr for nutr in nutrient_list if nutr != exclude]
    else:
        new_list = nutrient_list
    

    return compute_index(row, new_list, scaling_factor=scaling_factor)

In [12]:
def compute_qi_excluding_multiple(row, nutrient_list, exclude_list, scaling_factor=2000):

    new_list = nutrient_list.copy()  

    for nutr in exclude_list:
        if nutr in new_list:
            new_list.remove(nutr)

    return compute_index(row, new_list, scaling_factor=scaling_factor)

In [13]:
def compare_qi_excluding_nutrient(df, nutrient_to_exclude, qualifying_nutrients, new_col_name=None,scaling_factor=2000):


    if new_col_name is None:
        new_col_name = f"QI_excl_{nutrient_to_exclude}"


    df[new_col_name] = df.apply(lambda row: compute_qi_excluding(row, qualifying_nutrients, exclude=nutrient_to_exclude, scaling_factor=scaling_factor), axis=1)

 
    df_plot = df.drop_duplicates('combined_name').copy()
    df_plot = df_plot[['combined_name', 'QI', new_col_name]]

    labels = df_plot['combined_name'].tolist()
    x = np.arange(len(labels))
    width = 0.35


    plt.figure(figsize=(16, 8))
    plt.bar(x - width/2, df_plot['QI'], width, label='QI (incl. ' + nutrient_to_exclude + ')', color='skyblue')
    plt.bar(x + width/2, df_plot[new_col_name], width, label=f"QI (excl. {nutrient_to_exclude})", color='deeppink')

    plt.xlabel('Food Item')
    plt.ylabel('QI Value')
    plt.title(f"Comparison of QI with and without {nutrient_to_exclude}")
    plt.xticks(x, labels, rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.show()

    df_plot['QI_diff'] = df_plot['QI'] - df_plot[new_col_name]
    df_plot['QI_pct_change'] = (df_plot['QI_diff'] / df_plot['QI']) * 100

    return df_plot

In [14]:
def compute_nb(row, nutrient_cols, scaling_factor=2000) :
    truncated_ratios = []
    for nutr in nutrient_cols:
        ratio =   row[nutr + '_ratio_scaled']
        if ratio > 1.0 :
            ratio = 1
        truncated_ratios.append(ratio)

    nb_value =  (sum(truncated_ratios) / len(nutrient_cols)) * 100
    return nb_value

In [15]:
def compute_nb_excluding(row, nutrient_list, exclude=None, scaling_factor=2000):

    if exclude is not None:
        new_list = [nutr for nutr in nutrient_list if nutr != exclude]
    else:
        new_list = nutrient_list
    return compute_nb(row, new_list, scaling_factor=scaling_factor)


In [16]:
def compare_nb_excluding_nutrient(df, nutrient_to_exclude, qualifying_nutrients, new_col_name=None, scaling_factor=2000):
  
    if new_col_name is None:
        new_col_name = f"NB_excl_{nutrient_to_exclude}"

    df['NB'] = df.apply(lambda row: compute_nb(row, qualifying_nutrients, scaling_factor=scaling_factor), axis=1)

    df[new_col_name] = df.apply(lambda row: compute_nb_excluding(row, qualifying_nutrients, exclude=nutrient_to_exclude, scaling_factor=scaling_factor), axis=1)
    
    df_plot = df.drop_duplicates('combined_name').copy()
    df_plot = df_plot[['combined_name', 'NB', new_col_name]]
    
    labels = df_plot['combined_name'].tolist()
    x = np.arange(len(labels))
    width = 0.35
    
    plt.figure(figsize=(16, 8))
    plt.bar(x - width/2, df_plot['NB'], width, label='NB (incl. all)', color='skyblue')
    plt.bar(x + width/2, df_plot[new_col_name], width, label=f"NB (excl. {nutrient_to_exclude})", color='deeppink')
    plt.xlabel('Food Item')
    plt.ylabel('NB Value (%)')
    plt.title(f"Comparison of NB with and without {nutrient_to_exclude}")
    plt.xticks(x, labels, rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    df_plot['NB_diff'] = df_plot['NB'] - df_plot[new_col_name]
    df_plot['NB_pct_change'] = (df_plot['NB_diff'] / df_plot['NB']) * 100
    return df_plot


In [17]:
def compute_nb_excluding_multiple(row, nutrient_list, exclude_list, scaling_factor=2000):
    new_list = nutrient_list.copy()  
    for nutr in exclude_list:
        if nutr in new_list:
            new_list.remove(nutr)
    return compute_nb(row, new_list, scaling_factor=scaling_factor)

In [18]:
def interactive_exclude_nutrient(nutrient):
    clear_output(wait=True)
    df_plot = compare_qi_excluding_nutrient(
        df=high_qi_vegetable_fruit_juices, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )
    display(df_plot[['combined_name', 'QI', f"QI_excl_{nutrient}", 'QI_diff', 'QI_pct_change']])

    print(f"\nExcluding {nutrient} for NB:")
    df_nb = compare_nb_excluding_nutrient(
        df=high_qi_vegetable_fruit_juices, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )
    display(df_nb[['combined_name', 'NB', f"NB_excl_{nutrient}", 'NB_diff', 'NB_pct_change']])

nutrient_choice = ['folate_eaten', 
                   'niacin_eaten', 
                   'pantothenic_acid_eaten', 
                   'vitamin_b2_eaten', 
                   'vitamin_b1_eaten', 
                   'all_trans_retinol_equivalents_activity_eaten', 
                   'beta_carotene_eaten', 
                   'vitamin_b12_eaten', 
                   'vitamin_b6_eaten', 
                   'vitamin_c_eaten', 
                   'vitamin_d_eaten', 
                   'vitamin_e_activity_eaten', 
                   'calcium_eaten', 
                   'iron_eaten', 
                   'magnesium_eaten', 
                   'phosphorus_eaten', 
                   'potassium_eaten', 
                   'zinc_eaten', 
                   'fiber_eaten', 
                   'protein_eaten']

In [19]:
interact(interactive_exclude_nutrient, nutrient=widgets.Dropdown(options=nutrient_choice, description='Exclude:'))

interactive(children=(Dropdown(description='Exclude:', options=('folate_eaten', 'niacin_eaten', 'pantothenic_a…

<function __main__.interactive_exclude_nutrient(nutrient)>

- beta carotene
- vitamin c
- vitamin e

In [20]:
exclude_list = ['beta_carotene_eaten', 'vitamin_c_eaten', 'vitamin_e_eaten']
qualifying_nutrients = list(nutrient_info.keys())

for idx, row in high_qi_vegetable_fruit_juices.iterrows():

    qi_incl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=[])
    qi_excl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    nb_incl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=[]) 
    nb_excl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    print(f"Food item : {row['combined_name']}")
    print(f"QI (incl. all): {qi_incl}")
    print(f"QI (excl. {', '.join(exclude_list)}): {qi_excl}")
    print(f"NB (incl. all): {nb_incl:.2f}")
    print(f"NB (excl. {', '.join(exclude_list)}): {nb_excl:.2f}")
    print("-----------")

Food item : COOP NATURAPLAN BIO Biotta Gemüse-Cocktail
QI (incl. all): 23.80382058601723
QI (excl. beta_carotene_eaten, vitamin_c_eaten, vitamin_e_eaten): 2.7451442591906163
NB (incl. all): 76.28
NB (excl. beta_carotene_eaten, vitamin_c_eaten, vitamin_e_eaten): 74.31
-----------
Food item : Migros ORGANIC Vegetable Cocktail
QI (incl. all): 22.962530171071055
QI (excl. beta_carotene_eaten, vitamin_c_eaten, vitamin_e_eaten): 2.7554394313587185
NB (incl. all): 81.97
NB (excl. beta_carotene_eaten, vitamin_c_eaten, vitamin_e_eaten): 80.47
-----------
Food item : Jus de curcuma bio
QI (incl. all): 17.21767947155219
QI (excl. beta_carotene_eaten, vitamin_c_eaten, vitamin_e_eaten): 18.62795051759549
NB (incl. all): 66.55
NB (excl. beta_carotene_eaten, vitamin_c_eaten, vitamin_e_eaten): 69.64
-----------
Food item : Juice, vegetable
QI (incl. all): 17.148485223953262
QI (excl. beta_carotene_eaten, vitamin_c_eaten, vitamin_e_eaten): 1.8194689012259364
NB (incl. all): 74.03
NB (excl. beta_caroten

In [21]:
df_vegetable_fruit_juice[df_vegetable_fruit_juice['combined_name'] == 'Jus de curcuma bio'].drop_duplicates('combined_name')[[i for i in df_vegetable_fruit_juice.columns if '_ratio_scaled' in i]].T

,300997
folate_eaten_ratio_scaled,3.154574
niacin_eaten_ratio_scaled,6.083822
pantothenic_acid_eaten_ratio_scaled,6.813880
vitamin_b2_eaten_ratio_scaled,8.603384
vitamin_b1_eaten_ratio_scaled,3.049422
all_trans_retinol_equivalents_activity_eaten_ratio_scaled,0.000000
beta_carotene_eaten_ratio_scaled,0.000000
vitamin_b12_eaten_ratio_scaled,0.000000
vitamin_b6_eaten_ratio_scaled,5.338510
vitamin_c_eaten_ratio_scaled,0.588854


In [22]:
row = df_vegetable_fruit_juice[df_vegetable_fruit_juice['combined_name'] == 'Jus de curcuma bio'].iloc[0]

exclude_list = ['iron_eaten', 'fiber_eaten']
qualifying_nutrients = list(nutrient_info.keys())

qi_excl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list, scaling_factor=2000)
print(f"QI excluding {exclude_list}: {qi_excl:.2f}")

QI excluding ['iron_eaten', 'fiber_eaten']: 8.23
